In [1]:
import numpy as np
import pandas as pd
import scipy
import operator
import nltk
import os
import string
import copy
import copy
import pickle
import datetime
import joblib

import utils as my_utils

import multiprocessing
from scipy import spatial
from collections import Counter
from scipy.special import gammaln
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [2]:
dataset = pd.read_pickle("resources/electronics_glove_random_0.35_dataset")

In [3]:
dataset.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,text
35722,AKQOGUH6BOU9X,B000068O42,"Gary D. Smith ""hayman3347""","[0, 0]",Works great and the shipping was free. I use i...,5.0,GREAT!,1377648000,"08 28, 2013",work great ship free use connect guitar splitt...
571253,A3F9HQJCRL2984,B00285F22G,Joking85,"[0, 0]",I bought this mic as an alternative to my head...,5.0,A cheap solution mic,1363478400,"03 17, 2013",buy mic alternative headset game skyping doe f...


In [4]:
dataset.shape

(10000, 10)

In [ ]:
count_matrix, _, vocabulary, words = my_utils.processReviews(dataset['text'].values)


In [ ]:
# dataset = None

In [ ]:
n_top_words = 5

In [ ]:
topics_grid = [5, 10, 15, 20, 30, 40, 60, 100]

In [ ]:
folder_name = "baseline_lda_amazon_musical"

In [ ]:
folder_name = str(datetime.datetime.now()) + "_" + folder_name
os.mkdir("dumps/"+folder_name)

In [ ]:
n_cores = 35

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
def fit_model_multi(k):
    model = LatentDirichletAllocation(n_components=k, random_state=100, max_iter=100, n_jobs=n_cores, verbose=1)
    model.fit(count_matrix)
    joblib.dump(model, "dumps/" + folder_name + "/sampler_n_topics_" + str(k))
    print("done:", k)
    return model

In [ ]:
# pool = multiprocessing.Pool()
# models_dump = pool.map(fit_model_multi, topics_grid)
# pool.close()

In [ ]:
models_dump = []
for k in topics_grid:
    models_dump.append(fit_model_multi(k))

In [ ]:
for k, model in zip(topics_grid, models_dump):

    topic_words = {}
    for topic, comp in enumerate(model.components_):
        word_idx = np.argsort(comp)[::-1][:n_top_words]
        topic_words[topic] = [words[i] for i in word_idx]

    sample_df = []
    for topic, word in topic_words.items():
        sample_df.append(', '.join(word).split(", "))

    dt_distribution = model.transform(count_matrix)

#     print("\nK:", k)
#     print("Running Metrics...")
# #     print("H Score:", my_utils.get_hscore_multi(dt_distribution, count_matrix, k))
#     print("Log Likelihood:", model.score(count_matrix))
#     print("Perplexity:", model.perplexity(count_matrix))
#     print("Coherance Score:", my_utils.coherence_score(count_matrix, sample_df, vocabulary))
#     print("Silhouette Score:", silhouette_score(count_matrix, dt_distribution.argmax(axis=1)))
#     print("Davies Bouldin Score:", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1)))
    
    print my_utils.coherence_score(count_matrix, sample_df, vocabulary), "\t", silhouette_score(count_matrix, dt_distribution.argmax(axis=1)), "\t", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1))

# Appendix

In [ ]:
# def get_evaluations_multi(model):
#     topic_words = {}
#     for topic, comp in enumerate(model.components_):
#         word_idx = np.argsort(comp)[::-1][:n_top_words]
#         topic_words[topic] = [words[i] for i in word_idx]

#     sample_df = []
#     for topic, word in topic_words.items():
#         sample_df.append(', '.join(word).split(", "))

#     dt_distribution = model.transform(count_matrix)

#     h_score =  my_utils.get_hscore_multi(dt_distribution, count_matrix, k)
#     likelihood =  model.score(count_matrix)
#     perplexity = model.perplexity(count_matrix)
#     coherance_score = my_utils.coherence_score(count_matrix, sample_df, vocabulary)
#     silhouette = silhouette_score(count_matrix, dt_distribution.argmax(axis=1))
#     return [h_score, likelihood, perplexity, coherance_score, silhouette]

In [ ]:
# for k, model in zip(topics_grid, models_dump):

#     topic_words = {}
#     for topic, comp in enumerate(model.components_):
#         word_idx = np.argsort(comp)[::-1][:n_top_words]
#         topic_words[topic] = [words[i] for i in word_idx]

#     sample_df = []
#     for topic, word in topic_words.items():
#         sample_df.append(', '.join(word).split(", "))

#     dt_distribution = model.transform(count_matrix)

#     print("\nK:", k)
#     print("Running Metrics...")
#     print("Coherance Score:", my_utils.coherence_score(count_matrix, sample_df, vocabulary))
#     print("Silhouette Score:", silhouette_score(count_matrix, dt_distribution.argmax(axis=1)))
#     print("Davies Bouldin Score:", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1)))

In [ ]:
# print("Davies Bouldin Score:", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1)))

In [ ]:
#print("H-Score:", my_utils.get_hscore(dt_distribution, count_matrix, k))

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE

# X_embedded = TSNE(n_components=2).fit_transform(dt_distribution)

# X_embedded.shape

# plt.figure(figsize=(10, 10))
# plt.scatter([i[0] for i in X_embedded], [i[1] for i in X_embedded], c=dt_distribution.argmax(axis=1))